# Record Embedding

In [151]:
import pandas as pd
import numpy as np
import os
import calendar
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

## Embedding using fastText
Details here: https://fasttext.cc/

In [152]:
import warnings
warnings.filterwarnings('ignore')

import gensim
from gensim.models import FastText

Convert each tuple into a row

In [153]:
#hospital dataset

#1. load hospital dataset and turthvalue.csv; mark error for unmatched values and cleanup using FastText; fill empty cells with word2vec
#finding errors and outliers -> discover FDs and try marking errors? or word2vec most_similar of val and if row does not contain the cell values??!

df_hospital = pd.read_csv('dirty_data_transformed.csv', dtype=object)
df_truth = pd.read_csv('truthvalue.csv', dtype=object)



provider = df_truth['ProviderNumber'].tolist()
hospital = df_truth['HospitalName'].tolist()
address = df_truth['Address1'].tolist()
city = df_truth['City'].tolist()
state = df_truth['State'].tolist()
zipcode = df_truth['ZipCode'].tolist()
county = df_truth['CountyName'].tolist()
phone = df_truth['PhoneNumber'].tolist()
hospType = df_truth['HospitalType'].tolist()
owner = df_truth['HospitalOwner'].tolist()
service = df_truth['EmergencyService'].tolist()
condition = df_truth['Condition'].tolist()
code = df_truth['MeasureCode'].tolist()
name = df_truth['MeasureName'].tolist()
score = df_truth['Score'].tolist()
sample = df_truth['Sample'].tolist()
stateavg = df_truth['Stateavg'].tolist()

combined_hosp = list(zip(provider, hospital, address, city, state, zipcode, county, phone, hospType, owner, service, condition, code, name, score, sample, stateavg))



In [154]:
len(df_truth)

1000

In [155]:
len(combined_hosp[1])

17

In [76]:
#hospital dirty dataset; maintain dirty dataset separately
provider = df_hospital['ProviderNumber'].tolist()
hospital = df_hospital['HospitalName'].tolist()
address = df_hospital['Address1'].tolist()
city = df_hospital['City'].tolist()
state = df_hospital['State'].tolist()
zipcode = df_hospital['ZipCode'].tolist()
county = df_hospital['CountyName'].tolist()
phone = df_hospital['PhoneNumber'].tolist()
hospType = df_hospital['HospitalType'].tolist()
owner = df_hospital['HospitalOwner'].tolist()
service = df_hospital['EmergencyService'].tolist()
condition = df_hospital['Condition'].tolist()
code = df_hospital['MeasureCode'].tolist()
name = df_hospital['MeasureName'].tolist()
score = df_hospital['Score'].tolist()
sample = df_hospital['Sample'].tolist()
stateavg = df_hospital['Stateavg'].tolist()

combined_dirty = list(zip(provider, hospital, address, city, state, zipcode, county, phone, hospType, owner, service, condition, code, name, score, sample, stateavg))
len(combined_dirty)




1000

In [156]:
#Calculating error or dirty cells
dirty_list = []
truth_list = []
for i in range(len(combined_dirty)):
    for j in range(len(combined_dirty[0])):
        if combined_dirty[i][j] != combined_hosp[i][j]:
            dirty_list.append(combined_dirty[i][j])
            truth_list.append(combined_hosp[i][j])

In [157]:
len(combined_dirty)

1000

In [158]:
#training the model with truth data.
model_hosp = FastText(combined_hosp, min_count=1, workers=8, iter=100)

In [159]:
#module to predict the top value by the model.Some values are not broken into tokens.
#Values with keyerrors are stored in excluded_list

excluded_list = []
result = []
for i in range(len(dirty_list)):
    if isinstance(dirty_list[i], str):
        try:
            result.append((model_hosp.most_similar(dirty_list[i])[0][0],dirty_list[i],truth_list[i]))
        except KeyError:
            excluded_list.append(dirty_list[i])
            #print(dirty_list[i])
            pass

In [160]:
print (len(excluded_list))
print (len(result))
print (len(excluded_list) + len(result))
print (len(dirty_list))
excluded_list[0]


51
543
594
594


'Yxs'

In [161]:
#identify the true positives and false positives. Store them in the seperate list for further analysis.
true_pos = 0
false_pos = 0
correctly_predicted = []
falsely_predicted = []
for i in range(len(result)):
    if result[i][0] == result[i][2]:
        true_pos = true_pos + 1
        correctly_predicted.append(result[i])
    else:
        false_pos = false_pos + 1
        falsely_predicted.append(result[i])




In [139]:
print(len(result))
print(len(correctly_predicted))
print(len(falsely_predicted))

543
369
174


In [142]:
correctly_predicted[8]

for i in falsely_predicted:
    print (i)

('SHEFFIELD', '1xx19', '10019')
('588 patients', '3x patients', '37 patients')
('BOAZ', 'x5957', '35957')
('BOAZ', 'x0005', '10005')
('AL_PN-6', 'AL_PN-xB', 'AL_PN-3B')
('10009', '1000x', '10006')
('2059344011', '619 paxienxs', '619 patients')
('HF-4', 'HFx1', 'HF-1')
('AK_AMI-2', 'AxI-2', 'AMI-2')
('0%', 'x00%', '100%')
('1016 patients', '21 pxtients', '21 patients')
('36078', '360x9', '36049')
('MORGAN', '3x640', '35640')
('MORGAN', '3x640', '35640')
('HARTSELLE', '201 PINE SxREEx NORxHWESx', '201 PINE STREET NORTHWEST')
('588 patients', 'xx patients', '11 patients')
('48 patients', '6xpatients', '6 patients')
('HF-4', 'HF-x', 'HF-2')
('65%', 'x5%', '95%')
('AK_SCIP-INF-2', 'AL_SxIP-INF-2', 'AL_SCIP-INF-2')
('Voluntary non-profit - Other', 'Voluntary non-profit - Private', '10011')
('35653', 'Yes', "ST VINCENT'S EAST")
('Heart Attack Patients Given ACE Inhibitor or ARB for Left Ventricular Systolic Dysfunction (LVSD)', 'Surgical Infection Prevention', '50 MEDICAL PARK EAST DRIVE')
('

In [162]:
for i in correctly_predicted:
    print (i)

('surgery patients who were taking heart drugs called beta blockers before coming to the hospital who were kept on the beta blockers during the period just before and after their surgery', 'surgery patients who were taking heart drugs caxxed beta bxockers before coming to the hospitax who were kept on the beta bxockers during the period just before and after their surgery', 'surgery patients who were taking heart drugs called beta blockers before coming to the hospital who were kept on the beta blockers during the period just before and after their surgery')
('BIRMINGHAM', 'BIRMINGHxM', 'BIRMINGHAM')
('surgery patients needing hair removed from the surgical area before surgery who had hair removed using a safer method (electric clippers or hair removal cream �C not a razor)', 'surgery patients needing hair removed from the surgical area before surgery who had hair removed using a safer method (electric clippers or hair removal cream C not a razor)', 'surgery patients needing hair remov

In [163]:
falsely_predicted[1]

('588 patients', '3x patients', '37 patients')

In [164]:
model_hosp.most_similar('3595x')

[('35957', 0.9884520769119263),
 ('35976', 0.9203376770019531),
 ('BOAZ', 0.8603140115737915),
 ('2565938310', 0.8538941144943237),
 ('MARSHALL', 0.8307958841323853),
 ('2505 U S HIGHWAY 431 NORTH', 0.7380251884460449),
 ('2565718000', 0.7368029356002808),
 ('35960', 0.6880041360855103),
 ('MARSHALL MEDICAL CENTER NORTH', 0.6846861243247986),
 ('35903', 0.6814932823181152)]